# MEW via refining upper and lower bounds

#### Optimization approach
1. Quickly compute the upper and lower bounds of score of each candidate in all possible worlds
2. Delete any candidate whose upper bound is lower than another candidate's lower bound
3. For each voter:
    1. Compute the exact scores assigning to each candidate
    2. Refine the upper and lower bounds with these exact scores
    3. Delete any candidate whose upper bound is lower than another candidate's lower bound
    4. If there is only one candidate remained
        1. Declare this candidate to be the winner.
        2. Ignore the rest voters.
4. If the program didn't stop within the for loop, right now, all remaininig candidates have the same exact scores, and they are co-winners

#### Baseline approach
1. For each voter:
    1. Compute the exact scores assigning to each candidate
2. Declare the co-winners who have the highest score

#### Approach 2

1. Quickly compute the upper and lower bounds of score of each candidate in all possible worlds
2. Delete any candidate whose upper bound is lower than another candidate's lower bound
3. Top K optimization
    - LB_heap (higher to lower) - order of processing of candidates
    - UB_list (lower to higher) - order of pruning of candidates
    - exact scores -> UB_list
 

## Experiment setup

- Fix #candidates=10, change #voters [10, 100, 1000, 10000]
- Fix #voters=1000, change #candidates[5, 6, 7, 8, 9, 10]
- Use k-approval rule, where k = [1, 2, 3, 4]

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## Read data

__Caveat__: the posets are generated from uniformly random rankings, i.e., all candidates have the same winning probability during profile generation.

In [ ]:
df = pd.read_csv('experiment_output.tsv', sep='\t', comment='#')
df['speedup'] = df['t_baseline_sec'] / df['t_total_s']
df['skipped voters(%)'] = 100 - 100 * df['computed_voters'] / df['num_voters']
df['time of computing bounds (%)'] = 100 * df['t_quick_bounds_sec'] / df['t_total_s']
df.head()

# Overall

In [ ]:
plt.figure(dpi=120)
sns.boxplot(x='k_approval', y='speedup', data=df, linewidth=1, fliersize=1)
plt.xlabel('k (in k-approval)')
plt.title('In general, very few voters are skipped');
print('Conclusion: a key speedup factor is the value of k in k-approval.')

In [ ]:
plt.figure(dpi=120)
sns.histplot(df['skipped voters(%)'], bins=30);
plt.title('Few voters are skipped');
print('Conclusion: the dominant speedup origins from the candidates being pruned during voter enumeration.')

In [ ]:
plt.figure(dpi=120)
sns.histplot(df['time of computing bounds (%)'], bins=30);
print('Conclusion: in most cases, it is fast to compute upper and lower bounds.')

## Subset of data

Experiment setup

- Fix #candidates=10, change #voters [10, 100, 1000, 10000]
- Fix #voters=1000, change #candidates[5, 6, 7, 8, 9, 10]
- Use k-approval rule, where k = [1, 2, 3, 4]

In [ ]:
dfx = df.query('num_candidates == 10 and k_approval == 2')

plt.figure(dpi=120)
sns.boxplot(x='num_voters', y='speedup', data=dfx, linewidth=1, fliersize=1)
plt.xlabel('#voters')
print('Conclusion: speedup is less effective when increasing #voters.')

In [ ]:
dfx = df.query('num_voters == 1000 and k_approval == 2')

plt.figure(dpi=120)
sns.boxplot(x='num_candidates', y='speedup', data=dfx, linewidth=1, fliersize=1)
plt.xlabel('#candidates')
print('Conclusion: speedup is more effective when increasing #candidates.')